<a href="https://colab.research.google.com/github/MartynaPlomecka/DLM_DigitalFootprint/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Apr 16 18:16:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [12]:
## for data
import json
import pandas as pd
import numpy as np
from sklearn import metrics, manifold
## for processing
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for bert
!pip install transformers
import transformers
from transformers import AutoModelForMaskedLM, AutoTokenizer
from transformers import DataCollatorForLanguageModeling, DataCollatorForWholeWordMask
from transformers import Trainer, TrainingArguments
import sys
import gzip
from datetime import datetime

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
# mount to Google drive
from google.colab import drive
drive.mount('/content/drive')
# dataset paths
dataset_path = '/content/drive/My Drive/embeddings/table4model_clean.csv'
not_labelled_indeed_path = '/content/drive/My Drive/embeddings/indeed_intro_sh.csv'

import pandas as pd

df = pd.read_csv(dataset_path, header = 0)
df2 = pd.read_csv(not_labelled_indeed_path, header = 0)

Mounted at /content/drive


In [ ]:
#df2.values.tolist()

In [5]:
## create dtf
dtf = pd.DataFrame(df2)

In [6]:
## rename columns
dtf = dtf.rename(columns={ "Introduction":"text"})
## print 5 random rows
dtf.sample(5)

,text
1342,Patient-focused Registered Nurse with 20+ year...
3261,Experienced Associate Director of Human Resour...
3178,I am a motivated individual that is looking to...
470,My most extensive experience is working in the...
876,"5 years electrical apprentice, looking forward..."


In [7]:
'''
Preprocess indeed dataset BEFORE MLM
:parameter
    :param text: string - name of column containing text
    :param lst_stopwords: list - list of stopwords to remove
    :param flg_stemm: bool - whether stemming is to be applied
    :param flg_lemm: bool - whether lemmitisation is to be applied
:return
    cleaned text
'''
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()    
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

In [ ]:
lst_stopwords = nltk.corpus.stopwords.words("english")
lst_stopwords

In [9]:
dtf["text_clean"] = dtf["text"].apply(lambda x: 
          utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, 
          lst_stopwords=lst_stopwords))
dtf.head()

,text,text_clean
0,Highly skilled individual with fantastic energ...,highly skilled individual fantastic energy loo...
1,Ambitious professional who has completed cours...,ambitious professional completed coursework in...
2,"Experienced project and operations manager, wi...",experienced project operation manager localiza...
3,"* Highly organized, detail-oriented profession...",highly organized detailoriented professional e...
4,"Seeking a position in an office environment, w...",seeking position office environment need varie...


In [10]:
df2 = dtf[['text_clean']]

In [11]:
df2

,text_clean
0,highly skilled individual fantastic energy loo...
1,ambitious professional completed coursework in...
2,experienced project operation manager localiza...
3,highly organized detailoriented professional e...
4,seeking position office environment need varie...
...,...
3812,motivated personable professional experience w...
3813,resultoriented individual reputation strategic...
3814,certified six sigma master black belt 20 year ...
3815,obtain position successful growth oriented com...
